<a href="https://colab.research.google.com/github/JM3927/Hand-ASL-Estimation/blob/main/AI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://www.youtube.com/watch?v=a99p_fAr6e4
!git clone https://github.com/kinivi/hand-gesture-recognition-mediapipe.git

fatal: destination path 'hand-gesture-recognition-mediapipe' already exists and is not an empty directory.


In [4]:
!pip install opencv-python
!pip install mediapipe
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled p

In [3]:
# Get model from absolute path
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/latest/hand_landmarker.task

Importing Images - TRIM AMOUNT TO 10-20 PER FOLDER

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'asl-alphabet' dataset.
Path to dataset files: /kaggle/input/asl-alphabet


Image Scaling Via Pillow

In [6]:
# !pip install Pillow
from PIL import Image
import os

def resize_image(image_path, output_folder, factor):
    try:
        with Image.open(image_path) as img:
            new_size=(int(img.size[0]*factor),int(img.size[1]* factor))
            output_path=os.path.join(output_folder, os.path.basename(image_path))
            img = img.resize(new_size)
            img.save(output_path)
            print(f"Resized image of {new_size} saved to {output_path}")
    except Exception as e:
        print(f"Error resizing image{image_path}: {e}")

# def create_output_directory(path):
#     if not os.path.exists(path):
#         os.makedirs(path)

# def get_output_directory(input_directory):
#     relative_path = os.path.relpath(input_directory, input_dir)
#     return os.path.join(output_dir, relative_path)
# for subdir, dirs, files in os.walk(input_dir):
#     for file in files:
#         if file.endswith(".JPG"):
#             input_file_path = os.path.join(subdir, file)
#             output_directory=get_output_directory(subdir)
#             create_output_directory(output_directory)
#             output_file_path=os.path.join(output_directory, file)
#             if not os.path.exists(output_file_path):
#                 resize_image(input_file_path, output_directory, 0.5)

def grab_first_20_images(root):
    results = []

    for mainDir, subDirs, files in os.walk(root):
        # Take only the first 20 files in this folder
        files = sorted(files)
        top20 = files[:4]

        # Build full paths
        for fname in top20:
            results.append(os.path.join(mainDir, fname))

    return results

ASL_Path = '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'

# Creates unordered list of file paths
subdirList = grab_first_20_images(ASL_Path)
print(subdirList)

resizedImagesPath = "/content/resized_images"
os.makedirs(resizedImagesPath, exist_ok=True)

for path in subdirList:
    resize_image(path, resizedImagesPath, 1.28)

['/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/N/N1.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/N/N10.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/N/N100.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/N/N1000.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/R/R1.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/R/R10.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/R/R100.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/R/R1000.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/space/space1.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/space/space10.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/space/space100.jpg', '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/space/space1000.jpg', '/kaggle/input/asl-alphab

Running Through Base Model for Hand Pose Estimation

In [7]:
import csv
import mediapipe as mp

# List of letter and the list of coordinates for each image
def grab_all_files(root):
    results = []

    for mainDir, subDirs, files in os.walk(root):
        files = sorted(files)

        # Build full paths
        for fname in files:
            results.append(os.path.join(mainDir, fname))

    return results

imagesPath = '/content/resized_images'
fileList = grab_all_files(imagesPath)
print(len(fileList))
print(fileList)

# ex path: /content/resized_images/A1.jpg
# Wanted Output: A
folderNames = []

for path in fileList:
    ex = path.split('/')[-1]
    ex = ex.split('1')[0]
    # List of all group names
    if ex not in folderNames:
        folderNames.append(ex)

print(folderNames)
# Save data as .csv


/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


116
['/content/resized_images/A1.jpg', '/content/resized_images/A10.jpg', '/content/resized_images/A100.jpg', '/content/resized_images/A1000.jpg', '/content/resized_images/B1.jpg', '/content/resized_images/B10.jpg', '/content/resized_images/B100.jpg', '/content/resized_images/B1000.jpg', '/content/resized_images/C1.jpg', '/content/resized_images/C10.jpg', '/content/resized_images/C100.jpg', '/content/resized_images/C1000.jpg', '/content/resized_images/D1.jpg', '/content/resized_images/D10.jpg', '/content/resized_images/D100.jpg', '/content/resized_images/D1000.jpg', '/content/resized_images/E1.jpg', '/content/resized_images/E10.jpg', '/content/resized_images/E100.jpg', '/content/resized_images/E1000.jpg', '/content/resized_images/F1.jpg', '/content/resized_images/F10.jpg', '/content/resized_images/F100.jpg', '/content/resized_images/F1000.jpg', '/content/resized_images/G1.jpg', '/content/resized_images/G10.jpg', '/content/resized_images/G100.jpg', '/content/resized_images/G1000.jpg', '

In [14]:
# Passing file list through base model to obtain (mp needs python3.12 or earlier)
import cv2
from google.colab.patches import cv2_imshow
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import csv
import pandas as pd

# Test image
# !wget -q -O image.jpg https://storage.googleapis.com/mediapipe-tasks/hand_landmarker/woman_hands.jpg

base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=1)
detector = vision.HandLandmarker.create_from_options(options)

csvList = []
newFileList = []

# numDeleted = 0;

for i in range(len(fileList)):
# for i in range(1):
    imagePath = fileList[i]
    # image = mp.Image.create_from_file('/content/image.jpg')
    image = mp.Image.create_from_file(imagePath)

    detection_result = detector.detect(image)
    print(detection_result)

    # Iterate through detection_result.hand_landmarks and extract x, y coordinates
    if detection_result.hand_landmarks:
        newFileList.append(imagePath)
        # Gets the Letter Name of path (e.g., 'A', 'B', 'C')
        classLetter = (os.path.basename(imagePath).split('.')[0].split('1')[0])
        print(classLetter)

        # Map the class string to an integer label using the `folderNames` list
        try:
            class_label = folderNames.index(classLetter)
        except ValueError:
            print(f"Warning: Class '{classLetter}' from image '{imagePath}' not found in folderNames. Skipping this image.")
            continue # Skip this image if its class string cannot be mapped

        current_row_data = [class_label] # Start the row with the integer class label

        for hand_landmarks in detection_result.hand_landmarks:
            # Append all 21 keypoint values
            for landmark in hand_landmarks:
                current_row_data.append(landmark.x)
                current_row_data.append(landmark.y)

        csvList.append(current_row_data) # Add the completed row to csvList

# print(len(csvList))
# print(csvList)

with open("TrainingData.csv", "w+", newline="") as csv_file:
    writer = csv.writer(csv_file)
    # Write the data
    writer.writerows(csvList)

print(csvList)

HandLandmarkerResult(handedness=[[Category(index=0, score=0.8922438025474548, display_name='Right', category_name='Right')]], hand_landmarks=[[NormalizedLandmark(x=0.45479094982147217, y=0.5625106692314148, z=-2.705779991174495e-07, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5674633383750916, y=0.5184586048126221, z=-0.016595978289842606, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.6364245414733887, y=0.3862455487251282, z=-0.021394170820713043, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.661883533000946, y=0.26431453227996826, z=-0.02751394733786583, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.6388225555419922, y=0.18797877430915833, z=-0.030453916639089584, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5870346426963806, y=0.3209722936153412, z=-0.020092425867915154, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5878974795341492, y=0.2104366570711136, z=-0.062488265335559845, visibility=0.0, presence=0.0), NormalizedLandmark(x=0

Running Re-Training for mini NN

In [15]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

dataset = '/content/TrainingData.csv'
model_save_path = '/content/keypoint_classifier.keras'
tflite_save_path = '/content/keypoint_classifier.tflite'

NUM_CLASSES = 29

# Currently normalized on 0, prev. is normalized on input width/height
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)


model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

# Loading the saved model
model = tf.keras.models.load_model(model_save_path)
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_2 (Dropout)             │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 29)             │           319 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,389 (5.43 KB)

 Trainable params: 1,389 (5.43 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.0612 - loss: 3.3168
Epoch 1: saving model to /content/keypoint_classifier.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0612 - loss: 3.3168 - val_accuracy: 0.0588 - val_loss: 3.3694
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.0204 - loss: 3.3693
Epoch 2: saving model to /content/keypoint_classifier.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.0204 - loss: 3.3693 - val_accuracy: 0.0588 - val_loss: 3.3729
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.1020 - loss: 3.3053
Epoch 3: saving model to /content/keypoint_classifier.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.1020 - loss: 3.3053 - val_accuracy: 0.0588 - val_loss: 3.3764
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.0408 - loss: 3.3052
Epoch 4: saving model to /content/keypoint_classifier.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.0408 - loss: 3.3052 - val_a

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
